In [1]:
%load_ext autoreload
%autoreload 2  

In [2]:
from torchsummary import summary

In [3]:
import matplotlib.pyplot as plt 

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [5]:

import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from quickannotator.dl.inference import run_inference, getPendingInferenceTiles
from quickannotator.dl.dataset import TileDataset
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from safetensors.torch import save_file

def get_transforms(tile_size): #probably goes...elsewhere
    transforms = A.Compose([
    A.RandomScale(scale_limit=-.1, p=.1),
    A.PadIfNeeded(min_height=tile_size, min_width=tile_size),
    A.VerticalFlip(p=.5),
    A.HorizontalFlip(p=.5),
    A.Blur(p=.5),
    # # Downscale(p=.25, scale_min=0.64, scale_max=0.99),
    A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),
    # A.GridDistortion(p=.5, num_steps=5, distort_limit=(-0.3, 0.3),
    #                 border_mode=cv2.BORDER_REFLECT),
    # A.ISONoise(p=.5, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
    # A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
    # A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),
    # A.MultiplicativeNoise(p=.5, multiplier=(0.9, 1.1), per_channel=True, elementwise=True),
    # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=10, val_shift_limit=10, p=.9),
    A.Rotate(p=1, border_mode=cv2.BORDER_REFLECT),
    A.RandomCrop(tile_size, tile_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalization
    ToTensorV2()])
    return transforms

/opt/uv_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
classid = 2 
tile_size = 2_048 

boost_count = 5
batch_size_train=4
batch_size_infer=1
edge_weight=1_000
num_workers=0 

num_iters=10

dataset=TileDataset(classid, tile_size=tile_size, 
                    edge_weight=edge_weight, transforms=get_transforms(tile_size), 
                    boost_count=boost_count)

dataloader = DataLoader(dataset, batch_size=batch_size_train, shuffle=False, num_workers=num_workers) #NOTE: for dataset of type iter - shuffle must == False

model = smp.Unet(encoder_name="efficientnet-b0", encoder_weights="imagenet", 
                 decoder_channels=(64, 64, 64, 32, 16), in_channels=3, classes=1, encoder_freeze=True )

#model = model.half()

criterion = nn.BCEWithLogitsLoss(reduction='none', ).cuda()
optimizer = optim.NAdam(model.parameters(), lr=0.0001, weight_decay=1e-2) #TODO: this should be a setting

device = 'cuda' if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()



/tmp/ipykernel_128825/1762902026.py:27: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),


Unet(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding

In [7]:
for param in model.encoder.parameters():
    param.requires_grad = False
    
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"{name} is frozen")

encoder._conv_stem.weight is frozen
encoder._bn0.weight is frozen
encoder._bn0.bias is frozen
encoder._blocks.0._depthwise_conv.weight is frozen
encoder._blocks.0._bn1.weight is frozen
encoder._blocks.0._bn1.bias is frozen
encoder._blocks.0._se_reduce.weight is frozen
encoder._blocks.0._se_reduce.bias is frozen
encoder._blocks.0._se_expand.weight is frozen
encoder._blocks.0._se_expand.bias is frozen
encoder._blocks.0._project_conv.weight is frozen
encoder._blocks.0._bn2.weight is frozen
encoder._blocks.0._bn2.bias is frozen
encoder._blocks.1._expand_conv.weight is frozen
encoder._blocks.1._bn0.weight is frozen
encoder._blocks.1._bn0.bias is frozen
encoder._blocks.1._depthwise_conv.weight is frozen
encoder._blocks.1._bn1.weight is frozen
encoder._blocks.1._bn1.bias is frozen
encoder._blocks.1._se_reduce.weight is frozen
encoder._blocks.1._se_reduce.bias is frozen
encoder._blocks.1._se_expand.weight is frozen
encoder._blocks.1._se_expand.bias is frozen
encoder._blocks.1._project_conv.wei

In [ ]:
#_ = summary(model, (3, tile_size, tile_size),dtypes=[torch.float16]) #TODO: log this
_ = summary(model, (3, tile_size, tile_size)) #TODO: log this

Layer (type:depth-idx)                                  Output Shape              Param #
├─EfficientNetEncoder: 1-1                              [-1, 3, 2048, 2048]       --
|    └─Conv2dStaticSamePadding: 2-1                     [-1, 32, 1024, 1024]      --
|    |    └─ZeroPad2d: 3-1                              [-1, 3, 2049, 2049]       --
|    └─BatchNorm2d: 2-2                                 [-1, 32, 1024, 1024]      (64)
|    └─MemoryEfficientSwish: 2-3                        [-1, 32, 1024, 1024]      --
|    └─ModuleList: 2                                    []                        --
|    |    └─MBConvBlock: 3-2                            [-1, 16, 1024, 1024]      (1,448)
|    |    └─MBConvBlock: 3-3                            [-1, 24, 512, 512]        (6,004)
|    |    └─MBConvBlock: 3-4                            [-1, 24, 512, 512]        (10,710)
|    |    └─MBConvBlock: 3-5                            [-1, 40, 256, 256]        (15,350)
|    |    └─MBConvBlock: 3-6        

In [9]:
import datetime
running_loss = []
writer = SummaryWriter(log_dir=f"/tmp/{classid}/{datetime.datetime.now().strftime('%b%d_%H-%M-%S')}")

scaler = torch.amp.GradScaler("cuda")


for niter in tqdm(range(num_iters)): #TODO: this should be a setting
    images, masks, weights = next(iter(dataloader))
    #print ("post next iter")
    images = images.half().to(device)
    masks = masks.to(device)
    weights = weights.to(device)
    #print ("post copy ")
    epsilon = 1e-6
    for _ in tqdm(range(5),leave=False):
        
        with torch.autocast(device_type="cuda", dtype=torch.float16):
            outputs = model(images) 
            loss = criterion(outputs, masks.float())
            loss = (loss * (edge_weight ** weights).type_as(loss)).mean()

            positive_mask = (masks == 1).float()
            unlabeled_mask = (masks == 0).float()

            positive_loss = 1.0 * (loss * positive_mask).mean()
            unlabeled_loss = .1 * (loss * unlabeled_mask).mean()

            loss_total = positive_loss + unlabeled_loss
        
        #loss_total.backward()
        scaler.scale(loss_total).backward()

        #optimizer.step()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        
    running_loss.append(loss_total.item())

    writer.add_scalar(f'loss/loss', loss, niter)
    writer.add_scalar(f'loss/positive_loss', positive_loss, niter)
    writer.add_scalar(f'loss/unlabeled_loss', unlabeled_loss, niter)
    writer.add_scalar(f'loss/loss_total', loss_total, niter)
    
    #print ("losses:\t",loss_total,positive_mask.sum(),positive_loss,unlabeled_loss)

    
    if niter % 50==0:
        #print (f"niter [{niter}], Loss: {sum(running_loss)/len(running_loss)}")
        running_loss=[]

        #print ("saving!") #TODO: do we want to *always* override the last saved model , or do we want to instead only save if some type of loss threshold is met?
                            #another potentially more interesting option is to do both, save on a regular basis (since if we things crash we can revert back othe nearest checkpoint\
                            #but as well give the user in the front end a dropdown which enables them to select which model checkpoint they want to use? we had somethng similar in QAv1
                            #that said, this is likely a more advanced features and not very "apple like" since it would require explaining to the user when/why/how they should use the different models
                            #maybe suggest avoiduing for now --- lets just save the last one
        save_file(model.state_dict(), f"/tmp/model_{classid}.safetensors") #TODO: needs to go somewhere reasonable maybe /projid/models/classid/ ? or something
        last_save = 0



  0%|          | 0/10 [00:00<?, ?it/s]InitSpatiaMetaData() error:"table spatial_ref_sys already exists"
/opt/QuickAnnotator/quickannotator/db/utils.py:6: SAWarning: This declarative base already contains a class with the same class name and module name as quickannotator.db.utils.DynamicAnnotation, and will be replaced in the string-lookup table.
  class DynamicAnnotation(base):
  0%|          | 0/10 [00:02<?, ?it/s]


ValueError: Attempting to unscale FP16 gradients.

In [ ]:
images.shape

In [ ]:
tds=TileDataset(classid, tile_size=tile_size,
                    edge_weight=edge_weight, transforms=None, 
                    boost_count=boost_count)


In [ ]:
images, masks, weights= next(iter(tds))
plt.imshow(images)
plt.show()
plt.imshow(masks.squeeze())


In [ ]:
cv2.imwrite("mask2.png",masks.squeeze()*255)
cv2.imwrite("img2.png",images)

In [ ]:
images.shape

In [ ]:
masks.shape

In [ ]:
images, masks, weights = next(iter(dataloader))
i=images.cpu().detach()[0,::]
m=masks.cpu().detach()[0,::]
plt.imshow(i.squeeze().permute(1,2,0))
plt.show()
plt.imshow(m.squeeze())
print(masks.max())


In [ ]:
masks.max()

In [ ]:
plt.imshow(masks[0,::].cpu().detach().squeeze())  

In [ ]:
outputs = model(images.half().to(device))

In [ ]:
outputs

In [ ]:
o=torch.sigmoid(outputs.detach()).cpu()

In [ ]:
plt.imshow(o[0,::].squeeze()>.5)

In [ ]:
plt.hist(o.squeeze().numpy().flatten())

In [ ]:
masks[0,::].cpu().detach().squeeze()*255

In [ ]:
cv2.imwrite("gt_io.png",i.squeeze().permute(1,2,0).numpy()*128)

cv2.imwrite("gt.png",masks[0,::].cpu().detach().numpy().squeeze()*255)  
cv2.imwrite("out.png", (o[0,::].numpy().squeeze()>.5)*255)

In [ ]:
# import ttach as tta
# tta_model = tta.SegmentationTTAWrapper(model, tta.aliases.five_crop_transform(crop_height=2_016,crop_width=2_016), merge_mode='mean')
# outputs = tta_model(images.to(device))
# o=torch.sigmoid(outputs.detach()).cpu()
# plt.imshow(o[0,::].squeeze()>.5)

In [ ]:
masks.shape

In [ ]:
import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from quickannotator.dl.inference import run_inference, getPendingInferenceTiles
from quickannotator.dl.dataset import TileDataset
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from safetensors.torch import save_file

def get_transforms(tile_size): #probably goes...elsewhere
    transforms = A.Compose([
!    A.RandomScale(scale_limit=-0.1, p=.1),
!    A.PadIfNeeded(min_height=tile_size, min_width=tile_size),
!    A.VerticalFlip(p=.5),
!    A.HorizontalFlip(p=.5),
    # A.Blur(p=.5),
    # # Downscale(p=.25, scale_min=0.64, scale_max=0.99),
!    A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),
    # A.GridDistortion(p=.5, num_steps=5, distort_limit=(-0.3, 0.3),
    #                 border_mode=cv2.BORDER_REFLECT),
!    A.ISONoise(p=.5, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
    # A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
    # A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),
    # A.MultiplicativeNoise(p=.5, multiplier=(0.9, 1.1), per_channel=True, elementwise=True),
    # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=10, val_shift_limit=10, p=.9),
!    A.Rotate(p=1, border_mode=cv2.BORDER_REFLECT),
!    A.RandomCrop(tile_size, tile_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalization
    ToTensorV2()])
    return transforms

